In [1]:
%matplotlib inline

In [2]:
import pandas as pd
import gzip
import numpy as np
from scipy.optimize import minimize
from datetime import timedelta
import tensorflow as tf
from datetime import timedelta

In [3]:
df = pd.read_csv("/home/sathappan/workspace/time2event/hawkes/data/all_trades.csv")
df.t = pd.to_datetime(df.t)
pp = (df.t - df.t[0]).apply(lambda x: x/np.timedelta64(1, 'm')).value_counts()
events = []
e = 1.667e-6
for t, cnt in pp.iteritems():
    base_time = t
    for i in range(cnt):
        events.append(base_time + e*i)

events = np.array(sorted(events))
empirical = df.t
empirical_counts = empirical.value_counts().resample(rule='1min').sum()
events = events[None, :]


In [4]:
def calc_Ai(arrivals, mask, beta):
    tdiff = np.diff(arrivals) * mask[:, 1:]
    tdiff = np.exp(tdiff * -beta)
    A_i = np.zeros_like(arrivals, dtype=np.float128)
    for i in range(tdiff.shape[1]):
        #print(i,)
        A_i[:, i + 1] = tdiff[:, i] * (1 + A_i[:, i])
    return A_i, tdiff

In [5]:
mask = np.ones_like(events)
A_i, tempdiff = calc_Ai(events, mask, 1.0)

In [6]:
def calc_temporal_differences(arrivals, mask):
    temporal_diff = np.zeros((arrivals.shape[0], arrivals.shape[1], arrivals.shape[1]))
    #mask_tempdiff = np.zeros_like(temporal_diff)
    for t_i in range(arrivals.shape[1]):
        temporal_diff[:, :t_i, t_i] = arrivals[:, t_i] - arrivals[:, :t_i]
        #mask_tempdiff[:, :t_i, t_i] = 1.0
    mask_tempdiff = (temporal_diff != 0)
    
    return mask_tempdiff, temporal_diff
    ### Create a UsersxTimexTime matrix

In [7]:
mask_tdiff, temporal_diff = calc_temporal_differences(events, mask)

In [8]:
from scipy.sparse import csr_matrix

In [9]:
def loglikelihood():
    #tdiff = diff(arrivals) * mask[:, 1:]
    #A_i = tf.zeros_like(arrivals)
    #for i in range(tdiff.shape[0]):
    #    A_i[:, i + 1] = tdiff[:, i] * (1 + A[:, i ])
    #part1 = tf.log(mu + alpha * ais_tvar)
    #part1 = tf.reduce_sum(tf.log(mu + alpha * ais_tvar), axis=1)[:, None]
    hist_effects = tf.reduce_sum((tf.exp(-beta[:, :, None] * temp_diff_tvar))*mask_tdiff_tvar, axis=2)
    part1 = tf.reduce_sum(tf.log(mu + alpha * hist_effects), axis=1)
    #part1 = tf.reduce_sum()
    T_max = tf.reduce_max(arrivals_tvar*mask_tvar, axis=1)[:, None]
    part2 = mu * T_max
    tmp = tf.exp(-beta * ((T_max - arrivals_tvar)*mask_tvar))
    p3_tmp = tf.reduce_sum(tmp - tf.constant(1.0, dtype=tf.float64), axis=1)
    part3 = (alpha / beta)* p3_tmp[:, None]
    
    #return part1, part2, part3, part1 - part2 + part3
    return tf.reduce_sum(part1 - part2 + part3)

In [10]:
mask.shape

(1, 5000)

In [11]:
num_users = 1
alpha = tf.Variable(tf.random_uniform([num_users, 1], dtype=tf.float64), name='alpha')
#alpha = tf.Variable(tf.ones([num_users, 1], dtype=tf.float64), name='alpha')
beta = tf.Variable(tf.random_uniform([num_users, 1], dtype=tf.float64), name='beta')
#beta = tf.Variable(tf.ones([num_users, 1], dtype=tf.float64), name='beta')
#beta = tf.constant(1.0, dtype=tf.float64, name='beta')
mu = tf.Variable(tf.random_uniform([num_users, 1], dtype=tf.float64), name='mu')
#mu = tf.Variable(tf.ones([num_users, 1], dtype=tf.float64), name='mu')


In [12]:
T_max = 5000
arrivals_tvar = tf.placeholder(tf.float64, [None, T_max])
mask_tvar = tf.placeholder(tf.float64, [None, T_max])
temp_diff_tvar = tf.placeholder(tf.float64,[None, T_max, T_max])
mask_tdiff_tvar = tf.placeholder(tf.float64, [None, T_max, T_max])

In [13]:
def cost_func():
    nloglik = -loglikelihood()
    regularizer = tf.reduce_sum(alpha)  + tf.reduce_sum(mu) + tf.reduce_sum(alpha/beta) 
    return regularizer + nloglik

In [14]:
#p3 = loglikelihood()
p3 = cost_func()
#grad = tf.gradients(p3, [alpha, beta, mu])
optim = tf.train.GradientDescentOptimizer(0.01)

optim = tf.train.AdamOptimizer()

gr_alpha, gr_beta, gr_mu  = optim.compute_gradients(p3)
gr_alpha = (tf.clip_by_value(gr_alpha[0], 0.01, np.infty), gr_alpha[1])
gr_beta = (tf.clip_by_value(gr_beta[0], 0.01, np.infty), gr_beta[1])
gr_mu = (tf.clip_by_value(gr_mu[0], 0.01, np.infty), gr_mu[1])

minmzr = optim.apply_gradients([gr_alpha,gr_beta, gr_mu])
#p1, p2, p3 = loglikelihood()
#capped_gvs = [(tf.clip_by_value(grad, 0, 1), var) for grad, var in gvs]
#train_op = optim.apply_gradients(capped_gvs)

In [15]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    #tf_a = (sess.run([gr_alpha, minmzr], feed_dict={arrivals_tvar: np.nan_to_num(arrivals_numpy), mask_tvar:mask, ais_tvar:A_i, tdiff_tvar: tempdiff}))
    for i in range(1000):
        _, c, al, bt,m = (sess.run([minmzr, p3, alpha, beta, mu], feed_dict={arrivals_tvar: np.nan_to_num(events), mask_tvar:mask, temp_diff_tvar: temporal_diff, mask_tdiff_tvar:mask_tdiff}))
        if not (i % 50):
            print(c)
            print(al, bt, m)
    #tf_a, tf_b, tf_c, tf_d = (sess.run(p3, feed_dict={arrivals_tvar: np.nan_to_num(arrivals_numpy), mask_tvar:mask, ais_tvar:A_i, tdiff_tvar: tempdiff}))

-9055.07978182
[[ 0.60209926]] [[ 0.4505737]] [[ 0.55708193]]
-8952.45405836
[[ 0.55100284]] [[ 0.40057407]] [[ 0.50670238]]
-8843.60389464
[[ 0.49594542]] [[ 0.35057426]] [[ 0.45509425]]
-8731.54965131
[[ 0.43626932]] [[ 0.30057435]] [[ 0.40242578]]
-8620.45661281
[[ 0.37164104]] [[ 0.25057444]] [[ 0.3491288]]
-8519.86215932
[[ 0.30162264]] [[ 0.20057454]] [[ 0.2958621]]
-8449.65234506
[[ 0.22579191]] [[ 0.15057464]] [[ 0.24387888]]
-8438.17375232
[[ 0.14456756]] [[ 0.10057474]] [[ 0.1962149]]
-8388.72249724
[[ 0.06493126]] [[ 0.05057483]] [[ 0.16242246]]
-7044.914819
[[ 0.00599637]] [[ 0.00057492]] [[ 0.15713819]]
nan
[[-0.10212364]] [[-0.11353033]] [[ 0.10855909]]
nan
[[-0.23970336]] [[-0.25032562]] [[ 0.07448618]]
nan
[[-0.3799978]] [[-0.39022148]] [[ 0.04661738]]
nan
[[-0.52322477]] [[-0.53317933]] [[ 0.01922118]]
nan
[[-0.66927654]] [[-0.6790273]] [[ 0.00023067]]
nan
[[-0.81798138]] [[-0.82756785]] [[-0.02885874]]
nan
[[ nan]] [[ nan]] [[ nan]]
nan
[[ nan]] [[ nan]] [[ nan]]
nan


In [19]:
alpha.get_

<tf.Tensor 'alpha/read:0' shape=(1, 1) dtype=float64>

In [17]:
events[:, -1]

array([ 406.00000167])